# 📊 Proceso ETL - Análisis de Pedidos Entregados de Olist

## 🎯 Objetivo del Proyecto
Este notebook implementa un pipeline ETL (Extract, Transform, Load) robusto para consolidar múltiples fuentes de datos del ecosistema Olist en una tabla analítica unificada.

**Historia de Negocio:**
Olist es una plataforma de e-commerce brasileña que conecta pequeñas empresas con grandes marketplaces. Para maximizar el valor de los datos, necesitamos:
1. **Integrar** información dispersa en 6 datasets diferentes
2. **Filtrar** únicamente pedidos completados exitosamente (status='delivered')
3. **Generar** una tabla maestra que permita análisis estratégicos

**Resultado esperado:** Un dataset consolidado que responda preguntas como:
- ¿Qué categorías de productos generan mejores reviews?
- ¿Cuál es la preferencia de pago por estado?
- ¿Cómo se relaciona el precio con la satisfacción del cliente?

---

## 📦 Paso 1: Importación de Librerías

Utilizamos las herramientas esenciales del ecosistema Python para data engineering:
- **pandas**: Manipulación y transformación de datos tabulares
- **numpy**: Operaciones numéricas optimizadas
- **warnings**: Gestión de alertas durante el proceso

In [1]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime

# Configuración del entorno
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)  # Visualizar todas las columnas
pd.set_option('display.width', None)        # Sin límite de ancho

print("✅ Librerías cargadas exitosamente")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Librerías cargadas exitosamente
📅 Fecha de ejecución: 2026-01-21 14:09:31


## 🔄 Paso 2: EXTRACT - Carga de Datos Fuente

**Estrategia de Extracción:**
Cargamos 6 datasets críticos del ecosistema Olist. Cada archivo representa una dimensión del negocio:

| Dataset | Rol en el Negocio | Granularidad |
|---------|-------------------|-------------|
| Orders | Transacciones principales | 1 fila = 1 pedido |
| Customers | Datos demográficos | 1 fila = 1 cliente |
| Order Items | Detalle de productos | 1 fila = 1 ítem en pedido |
| Payments | Información financiera | 1 fila = 1 pago (puede haber múltiples) |
| Reviews | Satisfacción del cliente | 1 fila = 1 reseña |
| Products | Catálogo de productos | 1 fila = 1 producto único |

In [2]:
# Definición de rutas de archivos
archivos = {
    'orders': 'olist_orders_dataset.csv',
    'customers': 'olist_customers_dataset.csv',
    'order_items': 'olist_order_items_dataset.csv',
    'payments': 'olist_order_payments_dataset.csv',
    'reviews': 'olist_order_reviews_dataset.csv',
    'products': 'olist_products_dataset.csv'
}

# Carga de datasets con manejo de errores
dataframes = {}

for nombre, archivo in archivos.items():
    try:
        df = pd.read_csv(archivo)
        dataframes[nombre] = df
        print(f"✅ {nombre:15} | Filas: {len(df):,} | Columnas: {len(df.columns)}")
    except FileNotFoundError:
        print(f"❌ ERROR: No se encontró el archivo {archivo}")
    except Exception as e:
        print(f"❌ ERROR en {archivo}: {str(e)}")

print(f"\n📊 Total de datasets cargados: {len(dataframes)}/6")

✅ orders          | Filas: 99,441 | Columnas: 8
✅ customers       | Filas: 99,441 | Columnas: 5
✅ order_items     | Filas: 112,650 | Columnas: 7
✅ payments        | Filas: 103,886 | Columnas: 5
✅ reviews         | Filas: 99,224 | Columnas: 7
✅ products        | Filas: 32,951 | Columnas: 9

📊 Total de datasets cargados: 6/6


## 🔍 Paso 3: Exploración Inicial de Datos

Antes de transformar, entendemos la estructura y calidad de nuestros datos.

In [3]:
# Análisis de calidad de datos
print("=" * 80)
print("📋 REPORTE DE CALIDAD DE DATOS")
print("=" * 80)

for nombre, df in dataframes.items():
    print(f"\n🔹 {nombre.upper()}")
    print(f"   Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
    print(f"   Memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    # Valores nulos
    nulos = df.isnull().sum()
    if nulos.sum() > 0:
        print(f"   ⚠️  Valores nulos detectados:")
        for col, count in nulos[nulos > 0].items():
            print(f"      - {col}: {count:,} ({count/len(df)*100:.1f}%)")
    else:
        print(f"   ✅ Sin valores nulos")

# Vista previa del dataset de orders (crítico para filtrado)
print("\n" + "=" * 80)
print("👀 DISTRIBUCIÓN DE STATUS EN ORDERS")
print("=" * 80)
print(dataframes['orders']['order_status'].value_counts())
print(f"\n📌 Filtraremos únicamente pedidos con status = 'delivered'")

📋 REPORTE DE CALIDAD DE DATOS

🔹 ORDERS
   Dimensiones: 99,441 filas × 8 columnas
   Memoria: 58.97 MB
   ⚠️  Valores nulos detectados:
      - order_approved_at: 160 (0.2%)
      - order_delivered_carrier_date: 1,783 (1.8%)
      - order_delivered_customer_date: 2,965 (3.0%)

🔹 CUSTOMERS
   Dimensiones: 99,441 filas × 5 columnas
   Memoria: 29.62 MB
   ✅ Sin valores nulos

🔹 ORDER_ITEMS
   Dimensiones: 112,650 filas × 7 columnas
   Memoria: 39.43 MB
   ✅ Sin valores nulos

🔹 PAYMENTS
   Dimensiones: 103,886 filas × 5 columnas
   Memoria: 17.81 MB
   ✅ Sin valores nulos

🔹 REVIEWS
   Dimensiones: 99,224 filas × 7 columnas
   Memoria: 42.75 MB
   ⚠️  Valores nulos detectados:
      - review_comment_title: 87,656 (88.3%)
      - review_comment_message: 58,247 (58.7%)

🔹 PRODUCTS
   Dimensiones: 32,951 filas × 9 columnas
   Memoria: 6.79 MB
   ⚠️  Valores nulos detectados:
      - product_category_name: 610 (1.9%)
      - product_name_lenght: 610 (1.9%)
      - product_description_lenght:

## 🔧 Paso 4: TRANSFORM - Filtrado y Limpieza

**Decisión Estratégica de Negocio:**
Nos enfocamos en pedidos **entregados exitosamente** porque:
1. Representan transacciones completas (revenue real)
2. Tienen reviews válidas (feedback genuino)
3. Reflejan la experiencia completa del cliente

Pedidos cancelados, en proceso o con problemas se excluyen para mantener la integridad analítica.

In [4]:
# FILTRO 1: Orders con status 'delivered'
print("🔄 Aplicando filtro de status...\n")

orders_original = len(dataframes['orders'])
orders_delivered = dataframes['orders'][dataframes['orders']['order_status'] == 'delivered'].copy()
orders_filtrados = len(orders_delivered)

print(f"📊 Pedidos originales:  {orders_original:,}")
print(f"✅ Pedidos delivered:   {orders_filtrados:,}")
print(f"❌ Pedidos excluidos:   {orders_original - orders_filtrados:,}")
print(f"📈 Tasa de entrega:     {orders_filtrados/orders_original*100:.2f}%")

# Almacenar el dataset filtrado
dataframes['orders_delivered'] = orders_delivered

🔄 Aplicando filtro de status...

📊 Pedidos originales:  99,441
✅ Pedidos delivered:   96,478
❌ Pedidos excluidos:   2,963
📈 Tasa de entrega:     97.02%


## 🔗 Paso 5: TRANSFORM - Construcción de Tabla Maestra

**Arquitectura de Joins:**

```
                    ORDERS (delivered)
                          |
         +----------------+----------------+
         |                |                |
    CUSTOMERS        ORDER_ITEMS       REVIEWS
                          |
                     PRODUCTS
                          |
                      PAYMENTS
```

**Estrategia:**
- **Left joins** para preservar todos los pedidos delivered
- Agregación de payments (un pedido puede tener múltiples pagos)
- Agregación de reviews (priorizar la primera reseña por pedido)

In [5]:
# PASO 5.1: Preparación de datasets auxiliares
print("🔧 Preparando datasets para merge...\n")

# Agregación de PAYMENTS: Consolidar múltiples pagos por order_id
# Estrategia: Tomamos el tipo de pago más común (moda) por pedido
payments_agg = dataframes['payments'].groupby('order_id').agg({
    'payment_type': lambda x: x.mode()[0] if not x.mode().empty else x.iloc[0],
    'payment_value': 'sum'  # Suma total de pagos
}).reset_index()

print(f"✅ Payments agregado: {len(dataframes['payments']):,} registros → {len(payments_agg):,} pedidos únicos")

# Agregación de REVIEWS: Tomar la primera review por pedido
# Nota: En caso de múltiples reviews, priorizamos la primera cronológicamente
reviews_unique = dataframes['reviews'].sort_values('review_creation_date').groupby('order_id').first().reset_index()
reviews_unique = reviews_unique[['order_id', 'review_score']]

print(f"✅ Reviews procesado:  {len(dataframes['reviews']):,} registros → {len(reviews_unique):,} pedidos únicos")

🔧 Preparando datasets para merge...

✅ Payments agregado: 103,886 registros → 99,440 pedidos únicos
✅ Reviews procesado:  99,224 registros → 98,673 pedidos únicos


In [6]:
# PASO 5.2: Secuencia de Joins
print("\n🔗 Iniciando secuencia de joins...\n")

# JOIN 1: Orders + Customers (datos demográficos)
tabla_master = orders_delivered.merge(
    dataframes['customers'],
    on='customer_id',
    how='left'
)
print(f"✅ JOIN 1: Orders + Customers        | Filas: {len(tabla_master):,}")

# JOIN 2: + Order Items (productos en el pedido)
tabla_master = tabla_master.merge(
    dataframes['order_items'],
    on='order_id',
    how='left'
)
print(f"✅ JOIN 2: + Order Items             | Filas: {len(tabla_master):,}")

# JOIN 3: + Products (información de categoría)
tabla_master = tabla_master.merge(
    dataframes['products'][['product_id', 'product_category_name']],
    on='product_id',
    how='left'
)
print(f"✅ JOIN 3: + Products                | Filas: {len(tabla_master):,}")

# JOIN 4: + Reviews (satisfacción del cliente)
tabla_master = tabla_master.merge(
    reviews_unique,
    on='order_id',
    how='left'
)
print(f"✅ JOIN 4: + Reviews                 | Filas: {len(tabla_master):,}")

# JOIN 5: + Payments (método de pago)
tabla_master = tabla_master.merge(
    payments_agg[['order_id', 'payment_type']],
    on='order_id',
    how='left'
)
print(f"✅ JOIN 5: + Payments                | Filas: {len(tabla_master):,}")

print(f"\n🎉 Tabla maestra construida con éxito!")


🔗 Iniciando secuencia de joins...

✅ JOIN 1: Orders + Customers        | Filas: 96,478
✅ JOIN 2: + Order Items             | Filas: 110,197
✅ JOIN 3: + Products                | Filas: 110,197
✅ JOIN 4: + Reviews                 | Filas: 110,197
✅ JOIN 5: + Payments                | Filas: 110,197

🎉 Tabla maestra construida con éxito!


## ✨ Paso 6: Selección, Renombrado y Transformaciones Finales

Estandarizamos la estructura final y aplicamos transformaciones:
1. **Selección de columnas** relevantes del negocio
2. **Formato de fecha**: DD-MM-YYYY (sin hora)
3. **Numeración secuencial**: Columna order_number basada en fecha de compra
4. **Expansión de estados**: Nombres completos en lugar de siglas

In [7]:
# Selección de columnas relevantes según estructura objetivo
columnas_finales = [
    'order_id',
    'product_id',
    'price',
    'product_category_name',
    'order_purchase_timestamp',      # Será renombrado
    'order_status',                  # Será renombrado
    'customer_id',                   # Será renombrado
    'customer_unique_id',
    'customer_state',
    'review_score',
    'payment_type'
]

# Crear dataset final con columnas seleccionadas
tabla_final = tabla_master[columnas_finales].copy()

# Renombrado para coincidir con nomenclatura de tabla_principal
tabla_final.rename(columns={
    'order_purchase_timestamp': 'order_purchase_datetime',
    'order_status': 'orders_status',
    'customer_id': 'orders_customer_id'
}, inplace=True)

print("✅ Columnas seleccionadas y renombradas")

# TRANSFORMACIONES ADICIONALES
print("\n🔧 Aplicando transformaciones finales...")

# TRANSFORMACIÓN 1: Formato de Fecha (día-mes-año sin hora)
print("\n📅 1. Convirtiendo formato de fecha...")
tabla_final['order_purchase_datetime'] = pd.to_datetime(tabla_final['order_purchase_datetime'])
tabla_final['order_purchase_datetime'] = tabla_final['order_purchase_datetime'].dt.strftime('%d-%m-%Y')
print(f"   ✅ Fecha formateada a DD-MM-YYYY")

# TRANSFORMACIÓN 2: Crear columna order_number secuencial
print("\n🔢 2. Creando numeración secuencial de órdenes...")
temp_datetime = pd.to_datetime(tabla_final['order_purchase_datetime'], format='%d-%m-%Y')
order_fecha = tabla_final[['order_id']].copy()
order_fecha['fecha_temp'] = temp_datetime
order_ranking = order_fecha.drop_duplicates('order_id').sort_values('fecha_temp')
order_ranking['order_number'] = range(1, len(order_ranking) + 1)
order_map = dict(zip(order_ranking['order_id'], order_ranking['order_number']))
tabla_final['order_number'] = tabla_final['order_id'].map(order_map)
print(f"   ✅ {len(order_ranking):,} órdenes numeradas (1 a {len(order_ranking):,})")

# TRANSFORMACIÓN 3: Expandir nombres de estados brasileños
print("\n🗺️  3. Expandiendo nombres de estados...")
estados_brasil = {
    'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapa', 'AM': 'Amazonas',
    'BA': 'Bahia', 'CE': 'Ceara', 'DF': 'Distrito Federal',
    'ES': 'Espirito Santo', 'GO': 'Goias', 'MA': 'Maranhao',
    'MT': 'Mato Grosso', 'MS': 'Mato Grosso do Sul', 'MG': 'Minas Gerais',
    'PA': 'Para', 'PB': 'Paraiba', 'PR': 'Parana', 'PE': 'Pernambuco',
    'PI': 'Piaui', 'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul', 'RO': 'Rondonia', 'RR': 'Roraima',
    'SC': 'Santa Catarina', 'SP': 'Sao Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins'
}
tabla_final['customer_state'] = tabla_final['customer_state'].map(estados_brasil)
print(f"   ✅ {tabla_final['customer_state'].nunique()} estados convertidos")

# Reordenar columnas para que order_number esté al inicio
columnas_ordenadas = ['order_number'] + [col for col in tabla_final.columns if col != 'order_number']
tabla_final = tabla_final[columnas_ordenadas]

print("\n✅ Transformaciones completadas")
print(f"\n📋 Estructura final:")
print(tabla_final.dtypes)

✅ Columnas seleccionadas y renombradas

🔧 Aplicando transformaciones finales...

📅 1. Convirtiendo formato de fecha...
   ✅ Fecha formateada a DD-MM-YYYY

🔢 2. Creando numeración secuencial de órdenes...
   ✅ 96,478 órdenes numeradas (1 a 96,478)

🗺️  3. Expandiendo nombres de estados...
   ✅ 27 estados convertidos

✅ Transformaciones completadas

📋 Estructura final:
order_number                 int64
order_id                    object
product_id                  object
price                      float64
product_category_name       object
order_purchase_datetime     object
orders_status               object
orders_customer_id          object
customer_unique_id          object
customer_state              object
review_score               float64
payment_type                object
dtype: object


## 🧹 Paso 7: Validación de Calidad

Verificamos la calidad del dataset final con todas las transformaciones aplicadas.

In [8]:
# Reporte de calidad del dataset final
print("="*80)
print("🔍 VALIDACIÓN DE CALIDAD - DATASET FINAL")
print("="*80)

print(f"\n📊 Dimensiones: {tabla_final.shape[0]:,} filas × {tabla_final.shape[1]} columnas")
print(f"💾 Tamaño en memoria: {tabla_final.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# Verificar valores nulos
print(f"\n🔎 Análisis de completitud:")
nulos_finales = tabla_final.isnull().sum()
for col in tabla_final.columns:
    nulos = nulos_finales[col]
    pct_nulos = (nulos / len(tabla_final)) * 100
    status = "✅" if nulos == 0 else "⚠️"
    print(f"   {status} {col:30} | Nulos: {nulos:,} ({pct_nulos:.1f}%)")

# Verificar que todos los registros son 'delivered'
print(f"\n✅ Status únicos en el dataset: {tabla_final['orders_status'].unique()}")
print(f"✅ Total de pedidos delivered: {len(tabla_final):,}")

# Estadísticas descriptivas clave
print(f"\n📈 Insights del Dataset:")
print(f"   🛒 Productos únicos:  {tabla_final['product_id'].nunique():,}")
print(f"   👥 Clientes únicos:   {tabla_final['customer_unique_id'].nunique():,}")
print(f"   📦 Pedidos únicos:    {tabla_final['order_id'].nunique():,}")
print(f"   🗂️  Categorías:        {tabla_final['product_category_name'].nunique():,}")
print(f"   🌎 Estados:           {tabla_final['customer_state'].nunique():,}")
print(f"   ⭐ Review promedio:   {tabla_final['review_score'].mean():.2f}/5.0")

🔍 VALIDACIÓN DE CALIDAD - DATASET FINAL

📊 Dimensiones: 110,197 filas × 12 columnas
💾 Tamaño en memoria: 75.53 MB

🔎 Análisis de completitud:
   ✅ order_number                   | Nulos: 0 (0.0%)
   ✅ order_id                       | Nulos: 0 (0.0%)
   ✅ product_id                     | Nulos: 0 (0.0%)
   ✅ price                          | Nulos: 0 (0.0%)
   ⚠️ product_category_name          | Nulos: 1,537 (1.4%)
   ✅ order_purchase_datetime        | Nulos: 0 (0.0%)
   ✅ orders_status                  | Nulos: 0 (0.0%)
   ✅ orders_customer_id             | Nulos: 0 (0.0%)
   ✅ customer_unique_id             | Nulos: 0 (0.0%)
   ✅ customer_state                 | Nulos: 0 (0.0%)
   ⚠️ review_score                   | Nulos: 827 (0.8%)
   ⚠️ payment_type                   | Nulos: 3 (0.0%)

✅ Status únicos en el dataset: ['delivered']
✅ Total de pedidos delivered: 110,197

📈 Insights del Dataset:
   🛒 Productos únicos:  32,216
   👥 Clientes únicos:   93,358
   📦 Pedidos únicos:    96,478

## 💾 Paso 8: LOAD - Exportación del Dataset Final

Guardamos el dataset consolidado y transformado en formato CSV para análisis posteriores.

In [9]:
# Exportación a CSV con todas las transformaciones aplicadas
nombre_archivo = 'tabla_principal_etl.csv'

try:
    tabla_final.to_csv(nombre_archivo, index=False)
    print("="*80)
    print("💾 EXPORTACIÓN EXITOSA")
    print("="*80)
    print(f"\n✅ Archivo guardado: {nombre_archivo}")
    print(f"📊 Registros exportados: {len(tabla_final):,}")
    print(f"📁 Columnas totales: {len(tabla_final.columns)}")
    
    # Información sobre las transformaciones aplicadas
    print(f"\n🎨 Transformaciones Aplicadas:")
    print(f"   ✅ Fechas en formato: DD-MM-YYYY (sin hora)")
    print(f"   ✅ Orden secuencial: Numeración 1 a {tabla_final['order_number'].max():,}")
    print(f"   ✅ Estados: Nombres completos (sin abreviaciones)")
    
    # Vista previa enfocada en las transformaciones
    print(f"\n👀 Vista previa (primeras 10 filas con columnas clave):")
    columnas_preview = ['order_number', 'order_id', 'order_purchase_datetime', 
                        'customer_state', 'product_category_name', 'price']
    print(tabla_final[columnas_preview].head(10).to_string(index=False))
    
    print(f"\n📈 Estadísticas del Dataset Final:")
    print(f"   🔢 Órdenes únicas: {tabla_final['order_number'].nunique():,}")
    print(f"   📅 Rango de fechas: {tabla_final['order_purchase_datetime'].min()} a {tabla_final['order_purchase_datetime'].max()}")
    print(f"   🗺️  Estados únicos: {tabla_final['customer_state'].nunique()}")
    
except Exception as e:
    print(f"❌ ERROR al guardar el archivo: {str(e)}")

💾 EXPORTACIÓN EXITOSA

✅ Archivo guardado: tabla_principal_etl.csv
📊 Registros exportados: 110,197
📁 Columnas totales: 12

🎨 Transformaciones Aplicadas:
   ✅ Fechas en formato: DD-MM-YYYY (sin hora)
   ✅ Orden secuencial: Numeración 1 a 96,478
   ✅ Estados: Nombres completos (sin abreviaciones)

👀 Vista previa (primeras 10 filas con columnas clave):
 order_number                         order_id order_purchase_datetime      customer_state product_category_name  price
        26662 e481f51cbdc54678b7cc49136f2d6af7              02-10-2017           Sao Paulo utilidades_domesticas  29.99
        88456 53cdb2fc8bc7dce0b6741e2150273451              24-07-2018               Bahia            perfumaria 118.70
        92361 47770eb9100c2d0c44946d9cf07ec65d              08-08-2018               Goias            automotivo 159.90
        33657 949d5b44dbf5de918fe9c16f97b45f8a              18-11-2017 Rio Grande do Norte              pet_shop  45.00
        53505 ad21c59c0840e6cb83a9ceb5573f8159  

## 📊 Paso 9: Storytelling con Datos - Insights Estratégicos

**Del Dato a la Decisión:** Exploramos patrones que transforman información en acción.

### 🎯 Framework de Análisis
1. **¿Qué pasó?** (Descriptivo)
2. **¿Por qué pasó?** (Diagnóstico)
3. **¿Qué debemos hacer?** (Prescriptivo)

In [10]:
print("="*80)
print("📊 ANÁLISIS ESTRATÉGICO - INSIGHTS ACCIONABLES")
print("="*80)

# INSIGHT 1: Categorías Top por Volumen
print("\n🏆 TOP 10 CATEGORÍAS POR VOLUMEN DE VENTAS")
print("─"*80)
top_categorias = tabla_final['product_category_name'].value_counts().head(10)
for i, (cat, count) in enumerate(top_categorias.items(), 1):
    pct = (count / len(tabla_final)) * 100
    print(f"{i:2}. {cat:30} | {count:5,} pedidos ({pct:4.1f}%)")

print("\n💡 Recomendación: Estas categorías son el corazón del negocio.")
print("   → Priorizar stock y campañas de marketing en estas líneas.")
print("   → Analizar qué hace exitosas estas categorías para replicar en otras.")

📊 ANÁLISIS ESTRATÉGICO - INSIGHTS ACCIONABLES

🏆 TOP 10 CATEGORÍAS POR VOLUMEN DE VENTAS
────────────────────────────────────────────────────────────────────────────────
 1. cama_mesa_banho                | 10,953 pedidos ( 9.9%)
 2. beleza_saude                   | 9,465 pedidos ( 8.6%)
 3. esporte_lazer                  | 8,431 pedidos ( 7.7%)
 4. moveis_decoracao               | 8,160 pedidos ( 7.4%)
 5. informatica_acessorios         | 7,644 pedidos ( 6.9%)
 6. utilidades_domesticas          | 6,795 pedidos ( 6.2%)
 7. relogios_presentes             | 5,859 pedidos ( 5.3%)
 8. telefonia                      | 4,430 pedidos ( 4.0%)
 9. ferramentas_jardim             | 4,268 pedidos ( 3.9%)
10. automotivo                     | 4,140 pedidos ( 3.8%)

💡 Recomendación: Estas categorías son el corazón del negocio.
   → Priorizar stock y campañas de marketing en estas líneas.
   → Analizar qué hace exitosas estas categorías para replicar en otras.


In [11]:
# INSIGHT 2: Satisfacción del Cliente por Categoría
print("\n⭐ SATISFACCIÓN DEL CLIENTE (Review Score) POR CATEGORÍA")
print("─"*80)

satisfaccion = tabla_final.groupby('product_category_name').agg({
    'review_score': ['mean', 'count']
}).round(2)

satisfaccion.columns = ['review_promedio', 'num_reviews']
satisfaccion = satisfaccion[satisfaccion['num_reviews'] >= 50]  # Filtrar categorías con suficientes datos
satisfaccion = satisfaccion.sort_values('review_promedio', ascending=False)

print("\n🌟 TOP 5 CATEGORÍAS CON MAYOR SATISFACCIÓN (min. 50 reviews):")
for i, (cat, row) in enumerate(satisfaccion.head().iterrows(), 1):
    print(f"{i}. {cat:35} | ⭐ {row['review_promedio']:.2f} | 📊 {int(row['num_reviews'])} reviews")

print("\n⚠️  BOTTOM 5 CATEGORÍAS CON MENOR SATISFACCIÓN:")
for i, (cat, row) in enumerate(satisfaccion.tail().iterrows(), 1):
    print(f"{i}. {cat:35} | ⭐ {row['review_promedio']:.2f} | 📊 {int(row['num_reviews'])} reviews")

print("\n💡 Recomendación: ")
print("   → Investigar causas de baja satisfacción (calidad, envío, descripción).")
print("   → Implementar controles de calidad más estrictos en categorías problemáticas.")
print("   → Usar categorías top como benchmarks de excelencia operativa.")


⭐ SATISFACCIÓN DEL CLIENTE (Review Score) POR CATEGORÍA
────────────────────────────────────────────────────────────────────────────────

🌟 TOP 5 CATEGORÍAS CON MAYOR SATISFACCIÓN (min. 50 reviews):
1. livros_importados                   | ⭐ 4.51 | 📊 57 reviews
2. livros_interesse_geral              | ⭐ 4.51 | 📊 533 reviews
3. construcao_ferramentas_ferramentas  | ⭐ 4.44 | 📊 99 reviews
4. portateis_casa_forno_e_cafe         | ⭐ 4.44 | 📊 73 reviews
5. livros_tecnicos                     | ⭐ 4.39 | 📊 262 reviews

⚠️  BOTTOM 5 CATEGORÍAS CON MENOR SATISFACCIÓN:
1. casa_conforto                       | ⭐ 3.86 | 📊 427 reviews
2. audio                               | ⭐ 3.84 | 📊 358 reviews
3. fashion_roupa_masculina             | ⭐ 3.76 | 📊 124 reviews
4. telefonia_fixa                      | ⭐ 3.76 | 📊 252 reviews
5. moveis_escritorio                   | ⭐ 3.51 | 📊 1654 reviews

💡 Recomendación: 
   → Investigar causas de baja satisfacción (calidad, envío, descripción).
   → Implementar co

In [12]:
# INSIGHT 3: Preferencias de Pago por Estado
print("\n💳 MÉTODOS DE PAGO MÁS POPULARES")
print("─"*80)

pagos_total = tabla_final['payment_type'].value_counts()
for metodo, count in pagos_total.items():
    pct = (count / len(tabla_final)) * 100
    print(f"   {metodo:20} | {count:,} transacciones ({pct:.1f}%)")

# Análisis por estado (top 5 estados)
top_estados = tabla_final['customer_state'].value_counts().head(5).index

print("\n🗺️  PREFERENCIA DE PAGO POR ESTADO (Top 5 estados):")
for estado in top_estados:
    df_estado = tabla_final[tabla_final['customer_state'] == estado]
    metodo_preferido = df_estado['payment_type'].mode()[0]
    pct_metodo = (df_estado['payment_type'] == metodo_preferido).sum() / len(df_estado) * 100
    print(f"   {estado:5} | Prefiere: {metodo_preferido:15} ({pct_metodo:.1f}% de {len(df_estado):,} pedidos)")

print("\n💡 Recomendación:")
print("   → Optimizar proceso de checkout para el método de pago dominante.")
print("   → Considerar promociones específicas por método de pago en cada región.")


💳 MÉTODOS DE PAGO MÁS POPULARES
────────────────────────────────────────────────────────────────────────────────
   credit_card          | 84,154 transacciones (76.4%)
   boleto               | 22,362 transacciones (20.3%)
   voucher              | 2,026 transacciones (1.8%)
   debit_card           | 1,652 transacciones (1.5%)

🗺️  PREFERENCIA DE PAGO POR ESTADO (Top 5 estados):


   Sao Paulo | Prefiere: credit_card     (76.3% de 46,448 pedidos)
   Rio de Janeiro | Prefiere: credit_card     (79.1% de 14,143 pedidos)
   Minas Gerais | Prefiere: credit_card     (76.8% de 12,916 pedidos)
   Rio Grande do Sul | Prefiere: credit_card     (71.8% de 6,134 pedidos)
   Parana | Prefiere: credit_card     (74.5% de 5,649 pedidos)

💡 Recomendación:
   → Optimizar proceso de checkout para el método de pago dominante.
   → Considerar promociones específicas por método de pago en cada región.


In [13]:
# INSIGHT 4: Relación Precio - Satisfacción
print("\n💰 RELACIÓN PRECIO - SATISFACCIÓN DEL CLIENTE")
print("─"*80)

# Crear segmentos de precio
tabla_final['precio_segmento'] = pd.cut(
    tabla_final['price'], 
    bins=[0, 50, 100, 200, 500, float('inf')],
    labels=['0-50', '50-100', '100-200', '200-500', '500+']
)

precio_satisfaccion = tabla_final.groupby('precio_segmento').agg({
    'review_score': 'mean',
    'order_id': 'count'
}).round(2)

precio_satisfaccion.columns = ['review_promedio', 'num_pedidos']

print("\nSegmento de Precio | Review Promedio | Número de Pedidos")
print("─"*80)
for segmento, row in precio_satisfaccion.iterrows():
    print(f"   R$ {segmento:10} | ⭐ {row['review_promedio']:.2f}          | {int(row['num_pedidos']):,} pedidos")

correlacion = tabla_final[['price', 'review_score']].corr().iloc[0, 1]
print(f"\n📊 Correlación precio-satisfacción: {correlacion:.3f}")

if abs(correlacion) < 0.1:
    print("\n💡 Insight: El precio tiene POCA relación con la satisfacción del cliente.")
    print("   → La calidad del servicio y producto importa más que el precio.")
    print("   → Enfocarse en mejorar experiencia de entrega y calidad de producto.")
elif correlacion > 0:
    print("\n💡 Insight: A mayor precio, MAYOR satisfacción.")
    print("   → Los clientes asocian precio alto con mejor calidad.")
    print("   → Oportunidad de expandir líneas premium con valor añadido.")
else:
    print("\n💡 Insight: A mayor precio, MENOR satisfacción.")
    print("   → Revisar si los productos premium cumplen expectativas.")
    print("   → Ajustar estrategia de pricing o mejorar propuesta de valor.")


💰 RELACIÓN PRECIO - SATISFACCIÓN DEL CLIENTE
────────────────────────────────────────────────────────────────────────────────

Segmento de Precio | Review Promedio | Número de Pedidos
────────────────────────────────────────────────────────────────────────────────
   R$ 0-50       | ⭐ 4.08          | 38,530 pedidos
   R$ 50-100     | ⭐ 4.06          | 32,376 pedidos
   R$ 100-200    | ⭐ 4.11          | 26,356 pedidos
   R$ 200-500    | ⭐ 4.11          | 9,845 pedidos
   R$ 500+       | ⭐ 4.10          | 3,090 pedidos

📊 Correlación precio-satisfacción: 0.003

💡 Insight: El precio tiene POCA relación con la satisfacción del cliente.
   → La calidad del servicio y producto importa más que el precio.
   → Enfocarse en mejorar experiencia de entrega y calidad de producto.


In [14]:
# INSIGHT 5: Distribución Geográfica
print("\n🗺️  CONCENTRACIÓN GEOGRÁFICA DEL NEGOCIO")
print("─"*80)

dist_geografica = tabla_final['customer_state'].value_counts().head(10)
total_pedidos = len(tabla_final)

print("\nTop 10 Estados por Volumen de Pedidos:")
print("─"*80)
acumulado = 0
for i, (estado, count) in enumerate(dist_geografica.items(), 1):
    pct = (count / total_pedidos) * 100
    acumulado += pct
    print(f"{i:2}. {estado:5} | {count:6,} pedidos | {pct:5.1f}% | Acumulado: {acumulado:5.1f}%")

top3_pct = (dist_geografica.head(3).sum() / total_pedidos) * 100
print(f"\n📊 Los 3 estados principales concentran el {top3_pct:.1f}% del negocio")

print("\n💡 Recomendación:")
if top3_pct > 60:
    print("   ⚠️  ALTA CONCENTRACIÓN GEOGRÁFICA - Riesgo de dependencia regional.")
    print("   → Desarrollar estrategia de expansión a estados subrepresentados.")
    print("   → Investigar barreras de entrada en mercados desatendidos.")
else:
    print("   ✅ Distribución geográfica saludable.")
    print("   → Mantener estrategias de penetración en mercados secundarios.")


🗺️  CONCENTRACIÓN GEOGRÁFICA DEL NEGOCIO
────────────────────────────────────────────────────────────────────────────────

Top 10 Estados por Volumen de Pedidos:
────────────────────────────────────────────────────────────────────────────────
 1. Sao Paulo | 46,448 pedidos |  42.1% | Acumulado:  42.1%
 2. Rio de Janeiro | 14,143 pedidos |  12.8% | Acumulado:  55.0%
 3. Minas Gerais | 12,916 pedidos |  11.7% | Acumulado:  66.7%
 4. Rio Grande do Sul |  6,134 pedidos |   5.6% | Acumulado:  72.3%
 5. Parana |  5,649 pedidos |   5.1% | Acumulado:  77.4%
 6. Santa Catarina |  4,097 pedidos |   3.7% | Acumulado:  81.1%
 7. Bahia |  3,683 pedidos |   3.3% | Acumulado:  84.5%
 8. Distrito Federal |  2,355 pedidos |   2.1% | Acumulado:  86.6%
 9. Goias |  2,277 pedidos |   2.1% | Acumulado:  88.7%
10. Espirito Santo |  2,225 pedidos |   2.0% | Acumulado:  90.7%

📊 Los 3 estados principales concentran el 66.7% del negocio

💡 Recomendación:
   ⚠️  ALTA CONCENTRACIÓN GEOGRÁFICA - Riesgo de depend

## 🎓 Paso 10: Conclusiones y Próximos Pasos

### ✅ Lo que Logramos

1. **Pipeline ETL Robusto**: Integramos 6 fuentes de datos en una tabla analítica consolidada
2. **Calidad de Datos**: Filtrado preciso de pedidos delivered con validaciones completas
3. **Insights Accionables**: Identificamos oportunidades estratégicas basadas en datos

### 🔮 Siguientes Análisis Recomendados

**Análisis de Retención:**
- ¿Cuántos clientes son recurrentes vs. nuevos?
- ¿Qué categorías generan mayor lealtad?

**Análisis Temporal:**
- Tendencias de ventas por mes/día de la semana
- Estacionalidad por categoría de producto

**Análisis Predictivo:**
- Modelar probabilidad de review positivo (score >= 4)
- Predecir categorías con mayor potencial de crecimiento

**Optimización Operativa:**
- Análisis de tiempos de entrega por región
- Identificar cuellos de botella en la cadena logística

---

### 📚 Recursos y Documentación

**Archivos Generados:**
- `tabla_principal_etl.csv`: Dataset consolidado listo para análisis

**Dependencias:**
- pandas >= 1.3.0
- numpy >= 1.21.0

**Nota Técnica:** Este notebook está optimizado para datasets de tamaño mediano (< 10M registros). Para volúmenes mayores, considerar:
- Procesamiento por chunks con `pd.read_csv(chunksize=...)`
- Uso de Dask o PySpark para procesamiento distribuido
- Almacenamiento en formato Parquet para mayor eficiencia

In [15]:
# Resumen final de ejecución
print("="*80)
print("🎉 PROCESO ETL COMPLETADO CON ÉXITO")
print("="*80)
print(f"\n✅ Dataset final exportado: tabla_principal_etl.csv")
print(f"✅ Total de registros: {len(tabla_final):,}")
print(f"✅ Columnas incluidas: {len(tabla_final.columns)}")
print(f"✅ Status verificado: 100% delivered")
print(f"\n💾 El archivo está listo para análisis avanzados, visualizaciones y modelado predictivo.")
print(f"\n📅 Proceso finalizado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

🎉 PROCESO ETL COMPLETADO CON ÉXITO

✅ Dataset final exportado: tabla_principal_etl.csv
✅ Total de registros: 110,197
✅ Columnas incluidas: 13
✅ Status verificado: 100% delivered

💾 El archivo está listo para análisis avanzados, visualizaciones y modelado predictivo.

📅 Proceso finalizado: 2026-01-21 14:09:53
